In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import preprocess

## download 된 데이터로 DataFrame 생성 

In [2]:
df = preprocess.makeDataFrameFromDataSet(1.5*(10**6))
df_test = df[10**6:]
df = df[:10**6]
df = preprocess.makeTimeFeature(df)
df_test = preprocess.makeTimeFeature(df_test)

## Data Preprocess - 범주형 데이터 결측값 처리

In [3]:
ohe_columns = preprocess.ohe_columns
save_mode = pd.DataFrame(columns=['max_value','total','max_index'])

for ohe_column in ohe_columns :
    modeResult = preprocess.getMode(df[ohe_column])
    save_mode.loc[ohe_column] = modeResult
    if modeResult[0]/modeResult[1] >= 0.5 :
        df[ohe_column] = df[ohe_column].map(lambda x : modeResult[2] if x==str(-1) else x)

##서버에서 사용하기 위해 저장
save_mode.to_csv("./save_mode.csv",header=True,index=True)

## Croess Term 피쳐 생성

In [4]:
preprocess.makeCrossTermFeature(df,'product_country','hour')
preprocess.makeCrossTermFeature(df,'product_country','weekday')
preprocess.makeCrossTermFeature(df,'device_type','product_gender')

,Sale,SalesAmountInEuro,time_delay_for_conversion,click_timestamp,nb_clicks_1week,product_price,product_age_group,device_type,audience_id,product_gender,...,weekday,year,month,day,hour,minute,second,product_country_hour,product_country_weekday,device_type_product_gender
0,0,-1.00,-1,1598891820,-1,0.00,4C90FD52FC53D2C1C205844CB69575AB,7E56C27BFF0305E788DA55A029EC4988,-1,-1,...,1,2020,09,01,01,37,00,57A1D462A03BD076E029CF9310C11FC5_01,57A1D462A03BD076E029CF9310C11FC5_1,7E56C27BFF0305E788DA55A029EC4988_-1
1,0,-1.00,-1,1598925284,0,0.00,4C90FD52FC53D2C1C205844CB69575AB,D7D1FB49049702BF6338894757E0D959,-1,1B491180398E2F0390E6A588B3BCE291,...,1,2020,09,01,10,54,44,57A1D462A03BD076E029CF9310C11FC5_10,57A1D462A03BD076E029CF9310C11FC5_1,D7D1FB49049702BF6338894757E0D959_1B491180398E2...
2,0,-1.00,-1,1598892568,-1,0.00,4C90FD52FC53D2C1C205844CB69575AB,7E56C27BFF0305E788DA55A029EC4988,-1,-1,...,1,2020,09,01,01,49,28,-1_01,-1_1,7E56C27BFF0305E788DA55A029EC4988_-1
3,0,-1.00,-1,1598891174,-1,0.00,4C90FD52FC53D2C1C205844CB69575AB,D7D1FB49049702BF6338894757E0D959,B0E6BD3ACC0707BAB31CA2C2B08E652F,-1,...,1,2020,09,01,01,26,14,-1_01,-1_1,D7D1FB49049702BF6338894757E0D959_-1
4,0,-1.00,-1,1598905245,-1,0.00,4C90FD52FC53D2C1C205844CB69575AB,7E56C27BFF0305E788DA55A029EC4988,-1,-1,...,1,2020,09,01,05,20,45,57A1D462A03BD076E029CF9310C11FC5_05,57A1D462A03BD076E029CF9310C11FC5_1,7E56C27BFF0305E788DA55A029EC4988_-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,1,55.52,1689,1597179878,114,27.51,4C90FD52FC53D2C1C205844CB69575AB,7E56C27BFF0305E788DA55A029EC4988,-1,-1,...,2,2020,08,12,06,04,38,57A1D462A03BD076E029CF9310C11FC5_06,57A1D462A03BD076E029CF9310C11FC5_2,7E56C27BFF0305E788DA55A029EC4988_-1
999996,0,-1.00,-1,1597203004,-1,0.00,4C90FD52FC53D2C1C205844CB69575AB,7E56C27BFF0305E788DA55A029EC4988,-1,-1,...,2,2020,08,12,12,30,04,57A1D462A03BD076E029CF9310C11FC5_12,57A1D462A03BD076E029CF9310C11FC5_2,7E56C27BFF0305E788DA55A029EC4988_-1
999997,1,102.00,692,1597205953,-1,100.00,4C90FD52FC53D2C1C205844CB69575AB,7E56C27BFF0305E788DA55A029EC4988,-1,-1,...,2,2020,08,12,13,19,13,57A1D462A03BD076E029CF9310C11FC5_13,57A1D462A03BD076E029CF9310C11FC5_2,7E56C27BFF0305E788DA55A029EC4988_-1
999998,0,-1.00,-1,1597164316,-1,0.00,4C90FD52FC53D2C1C205844CB69575AB,D7D1FB49049702BF6338894757E0D959,951795EB43464C86FB534742A58D99FB,-1,...,2,2020,08,12,01,45,16,57A1D462A03BD076E029CF9310C11FC5_01,57A1D462A03BD076E029CF9310C11FC5_2,D7D1FB49049702BF6338894757E0D959_-1


## DataPreprocess - One Hot Encoding

In [5]:
## 범주형 데이터 one hot encoding
df_preprocess = pd.DataFrame()

df_preprocess = preprocess.ohe(df,df_preprocess,use_saved_columns=False)

# 서버에서 사용하기 위해 칼럼들 저장
df_ohe_columns = pd.DataFrame(list(df_preprocess.columns))
df_ohe_columns.to_csv('saved_ohe_columns.csv',header=None)

## 정수형 데이터 결측값 처리

In [6]:
## 정수형 데이터 결측값 평균으로 처리
import preprocess
import pandas as pd

integer_columns = preprocess.integer_columns

for integer_column in integer_columns :
    df_preprocess = pd.concat([df['nb_clicks_1week'],df_preprocess],axis=1)

#nb_clicks_1week 의 -1 값을 모두 평균으로 대체
for integer_column in integer_columns :
    df_preprocess[integer_column][df_preprocess[integer_column]==-1] = df_preprocess[integer_column].mean()

#서버에서 사용하기 위해 저장
preprocess.save_value_by_csv('mean',df[integer_column].mean(),'df')

## Data Preprocess- std Normalization

In [7]:
# 서버에서 사용하기 위해 저장
preprocess.save_value_by_csv('mean',df_preprocess[integer_column].mean(),'df_preprocess')
preprocess.save_value_by_csv('std',df_preprocess[integer_column].std(),'df_preprocess')

In [8]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

integer_columns = preprocess.integer_columns
print(integer_columns)

# standard Normalization
preprocess.scale(df_preprocess,StandardScaler())

# 서버에서 사용하기 위해 저장
preprocess.save_value_by_csv('min',df_preprocess[integer_column].min(),'df_preprocess')
preprocess.save_value_by_csv('max',df_preprocess[integer_column].max(),'df_preprocess')

# minmax Normalization
preprocess.scale(df_preprocess,MinMaxScaler())

['nb_clicks_1week']


,nb_clicks_1week,device_type_0C297EB7365042A7DCB803298164359F,device_type_1D2373C2A02F5A307C1F0C697ECF8429,device_type_7E56C27BFF0305E788DA55A029EC4988,device_type_B432A885981287B47E08B41E802860B2,device_type_D7D1FB49049702BF6338894757E0D959,device_type_FF2C446555E3822B0E0FC3406116E86D,product_gender_-1,product_gender_0FB06F1EAC1E00A436B336C5DF3C14AF,product_gender_1B491180398E2F0390E6A588B3BCE291,...,device_type_product_gender_FF2C446555E3822B0E0FC3406116E86D_1B491180398E2F0390E6A588B3BCE291,device_type_product_gender_FF2C446555E3822B0E0FC3406116E86D_26FE89E9DD2E6FC18AA5BE1F6D5A6870,device_type_product_gender_FF2C446555E3822B0E0FC3406116E86D_3B3F1EFC68D245B766D311E45CD6B955,device_type_product_gender_FF2C446555E3822B0E0FC3406116E86D_6EFABCEDA36A931DBF760F88970BAF0E,device_type_product_gender_FF2C446555E3822B0E0FC3406116E86D_86E2AFBF909EC95B069893FF0BBC5B26,device_type_product_gender_FF2C446555E3822B0E0FC3406116E86D_8CE4C5DDEE22BB8EE50CF418ED81591C,device_type_product_gender_FF2C446555E3822B0E0FC3406116E86D_A5D15FC386510762EC0DDFF54ABE6F94,device_type_product_gender_FF2C446555E3822B0E0FC3406116E86D_C45A9AC6D102ACAEEDF0D6F78636D84A,device_type_product_gender_FF2C446555E3822B0E0FC3406116E86D_C83584C67375245584CEBF5DCE5BEE2D,device_type_product_gender_FF2C446555E3822B0E0FC3406116E86D_D7B42B0C4D807EF1C13F79948743E9DA
0,0.010744,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.000000,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0.010744,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0.010744,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0.010744,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,0.005786,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
999996,0.010744,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
999997,0.010744,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
999998,0.010744,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


## Tensorflow를 이용하여 모델 생성

In [9]:
l1_reg=preprocess.l1_reg

In [10]:
X = df_preprocess
Y = df['Sale'].astype(float)

In [11]:
import tensorflow as tf
from tensorflow.keras import layers

In [12]:
model = tf.keras.Sequential(name="my_sequential")
model.add(layers.Dense(1, activation="sigmoid",kernel_regularizer=l1_reg))
model.compile(optimizer="Adam", loss="binary_crossentropy", metrics=["AUC"])
model.fit(X,Y,batch_size=100,epochs=5)

Epoch 1/5
10000/10000 [==============================] - 7s 686us/step - loss: 0.3624 - auc: 0.6254
Epoch 2/5
10000/10000 [==============================] - 7s 674us/step - loss: 0.3350 - auc: 0.6628
Epoch 3/5
10000/10000 [==============================] - 7s 671us/step - loss: 0.3339 - auc: 0.6625
Epoch 4/5
10000/10000 [==============================] - 7s 666us/step - loss: 0.3341 - auc: 0.6613
Epoch 5/5
10000/10000 [==============================] - 7s 668us/step - loss: 0.3326 - auc: 0.6642


## 학습 성능 평가 및 결과값 저장

In [13]:
cvr_result = model.predict(df_preprocess)
filePath = 'train_result.txt'

preprocess.result_to_txt_file(df['Sale'],cvr_result,filePath)

In [14]:
model.evaluate(df_preprocess,Y,batch_size=100)

10000/10000 [==============================] - 7s 648us/step - loss: 0.3337 - auc: 0.6648


[0.33368244767189026, 0.6647629737854004]

## 모델 저장 및 이동

In [15]:
# 전체 모델을 저장
model.save('model.h5')

In [17]:
!mv model.h5 ./model/model.dat

## 평가 데이터 50만개 전처리 및 결과값 예측

In [18]:
df_test_preprocess = preprocess.totalProcess(df_test)
df_test['Sale'] = df_test['Sale'].astype(float)

## 성능 평가 및 데이터 저장

In [19]:
model_path = './model/model.dat'
model = tf.keras.models.load_model(model_path,custom_objects={'l1_reg': l1_reg})

In [20]:
model.evaluate(df_test_preprocess,df_test['Sale'],batch_size=100)

cvr_result =  model.predict(df_test_preprocess)
filePath = 'test_result.txt'

preprocess.result_to_txt_file(df_test['Sale'],cvr_result,filePath)

5000/5000 [==============================] - 3s 604us/step - loss: 0.3438 - auc: 0.6613


In [21]:
model.predict(df_preprocess[:1])

array([[0.09978476]], dtype=float32)